In [2]:
from google.cloud import storage, aiplatform
import tensorflow as tf

In [3]:
ARTIFACT_FILENAME = "model.pkl"
BUCKET = "iris-classification"
ARTIFACT_URI = f"gs://{BUCKET}"
MODEL = "iris"
ENDPOINT = MODEL + "_endpoint"

# the serving container image
# https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
DEPLOY_IMAGE = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

PROJECT = "cloud4us-gcp-3fbr5mcqluov7itt4"
LOCATION = "europe-west1"

# create Cloud Storage bucket

In [4]:
client = storage.Client()
bucket = client.bucket(BUCKET)
if not bucket.exists():
    new_bucket = client.create_bucket(bucket, location="europe-west1")

# copy local .pkl file to Cloud Storage

In [5]:
!gsutil cp {ARTIFACT_FILENAME} {ARTIFACT_URI}

Copying file://model.pkl [Content-Type=application/octet-stream]...
/ [1 files][174.2 KiB/174.2 KiB]                                                
Operation completed over 1 objects/174.2 KiB.                                    


# init aiplatform

In [6]:
aiplatform.init(project=PROJECT, location=LOCATION)

# upload model to Vertex AI

In [7]:
%%time

model = aiplatform.Model.upload(
    display_name=MODEL,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE
)

Creating Model
Create Model backing LRO: projects/704710201124/locations/europe-west1/models/7620424821045723136/operations/6881923542598811648
Model created. Resource name: projects/704710201124/locations/europe-west1/models/7620424821045723136@1
To use this Model in another session:
model = aiplatform.Model('projects/704710201124/locations/europe-west1/models/7620424821045723136@1')
CPU times: user 58 ms, sys: 27.1 ms, total: 85.1 ms
Wall time: 19.5 s


# deploy model to endpoint

In [8]:
%%time
endpoint = model.deploy()

Creating Endpoint
Create Endpoint backing LRO: projects/704710201124/locations/europe-west1/endpoints/4954610500991188992/operations/8386125818140557312
Endpoint created. Resource name: projects/704710201124/locations/europe-west1/endpoints/4954610500991188992
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/704710201124/locations/europe-west1/endpoints/4954610500991188992')
Deploying model to Endpoint : projects/704710201124/locations/europe-west1/endpoints/4954610500991188992
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/704710201124/locations/europe-west1/endpoints/4954610500991188992/operations/892136038196051968
Endpoint model deployed. Resource name: projects/704710201124/locations/europe-west1/endpoints/4954610500991188992
CPU times: user 254 ms, sys: 51 ms, total: 305 ms
Wall time: 12min 58s


# 1️⃣ online predictions (using endpoint)

In [9]:
instances = [
    [6.8, 3.2, 5.7, 2.3],
    [5.0 , 2.0 , 3.5, 1.0],
    [5.4, 3.4, 1.7, 0.2]
]

In [10]:
prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[2.0, 1.0, 0.0], deployed_model_id='3110245716094615552', model_version_id='1', model_resource_name='projects/704710201124/locations/europe-west1/models/7620424821045723136', explanations=None)

# 2️⃣ batch predictions (using model)

In [11]:
filename_1 = "iris.jsonl"
filename_2 = "iris_1.csv"
filename_3 = "iris_2.csv"

batch_name_1 = "batch-jsonl"
batch_name_2 = "batch-csv-1"
batch_name_3 = "batch-csv-2"

In [12]:
# Create files in Cloud Storage

def create_file(filename, data_string):
    blob = bucket.blob(filename)
    blob.upload_from_string(data_string)

data_string = """
[6.8, 3.2, 5.7, 2.3]
[5.0, 2.0, 3.5, 1.0]
[5.4, 3.4, 1.7, 0.2]
""".strip("\n")
create_file(filename_1, data_string)
    
data_string = """
"sepal length (cm)","sepal width (cm)","petal length (cm)","petal width (cm)"
6.8,3.2,5.7,2.3
5.0,2.0,3.5,1.0
5.4,3.4,1.7,0.2
""".strip("\n")
create_file(filename_2, data_string)

data_string = """
6.8,3.2,5.7,2.3
5.0,2.0,3.5,1.0
5.4,3.4,1.7,0.2
""".strip("\n")
create_file(filename_3, data_string)

In [13]:
def create_batch_prediction_job(filename, batch_name):
    batch_prediction_job = model.batch_predict(
        job_display_name=batch_name,
        gcs_source=ARTIFACT_URI + "/" + filename,
        gcs_destination_prefix=ARTIFACT_URI,
        machine_type="n1-standard-2"
    )
    return batch_prediction_job

In [14]:
%%time
batch_prediction_job_1 = create_batch_prediction_job(filename_1, batch_name_1)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/704710201124/locations/europe-west1/batchPredictionJobs/9102728123497054208
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/704710201124/locations/europe-west1/batchPredictionJobs/9102728123497054208')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/europe-west1/batch-predictions/9102728123497054208?project=704710201124
BatchPredictionJob projects/704710201124/locations/europe-west1/batchPredictionJobs/9102728123497054208 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/704710201124/locations/europe-west1/batchPredictionJobs/9102728123497054208 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/704710201124/locations/europe-west1/batchPredictionJobs/9102728123497054208 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/704710201124/locations/europe-west1/batchPredicti

In [23]:
# %%time
# batch_prediction_job_2 = create_batch_prediction_job(filename_2, batch_name_2)

In [24]:
# %%time
# batch_prediction_job_3 = create_batch_prediction_job(filename_3, batch_name_3)

In [25]:
def print_output(batch_prediction_job):
    bp_iter_outputs = batch_prediction_job.iter_outputs()
    
    explanation_results = list()
    for blob in bp_iter_outputs:
        explanation_results.append(blob.name)
    
    for explanation_result in explanation_results:
        gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
        with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
            for line in gfile.readlines():
                print(line)

In [26]:
print_output(batch_prediction_job_1)

{"instance": [6.8, 3.2, 5.7, 2.3], "prediction": 2}

{"instance": [5.0, 2.0, 3.5, 1.0], "prediction": 1}

{"instance": [5.4, 3.4, 1.7, 0.2], "prediction": 0}



In [27]:
# print_output(batch_prediction_job_2)

In [28]:
# print_output(batch_prediction_job_3)

# 🧹 [optional] clean up

In [30]:
# endpoint.delete(force=True)  # force=True to undeploy all models before deleting endpoint
# model.delete()
# !gcloud storage rm --recursive {ARTIFACT_URI}  # delete Cloud Storage bucket
# batch_prediction_job_1.delete()
# batch_prediction_job_2.delete()
# batch_prediction_job_3.delete()